In [1]:
import c3d
from c3d import *

feature_extractor = c3d_feature_extractor()

In [2]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm

X_tr=[]
labels = []
path = "dataset/Violence/"
random.seed(40)
listing2 = os.listdir(path)
for vid2 in tqdm(listing2[:500]):
    vid2 = path +vid2
    label = vid2.split("/")[1]
    frames = []
    cap = cv2.VideoCapture(vid2)    
    count = 0
    while True:
        ret, frame = cap.read()
        count += 3
        if frame is None:
            break
        frame=cv2.resize(frame,(240, 320)) 
        frames.append(frame)
        if len(frames) > 65:
            break
        cap.set(1, count)
        
    input=np.array(frames)
    clip = preprocess_input(input)
    rgb_feature = feature_extractor.predict(clip)[0]
    X_tr.append(rgb_feature)
    labels.append(label)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [12:54<00:00,  1.55s/it]


In [3]:
X_tr_1= np.array(X_tr)
print(X_tr_1.shape)

(500, 4096)


In [4]:
%%time
import os
import cv2
import numpy as np
import random


random.seed(40)
listing2 = os.listdir('dataset/NonViolence/')
for vid2 in tqdm(listing2[:500]):
    vid2 = 'dataset/NonViolence/'+vid2
    label = vid2.split("/")[1]
    frames = []
    cap = cv2.VideoCapture(vid2)    
    count = 0
    while True:
        ret, frame = cap.read()
        count += 3
        if frame is None:
            break
        frame=cv2.resize(frame,(240,320))  
        frames.append(frame)
        if len(frames) > 65:
            break        
        cap.set(1, count)   
    input=np.array(frames)
    clip = preprocess_input(input)
    rgb_feature = feature_extractor.predict(clip)[0]
    X_tr.append(rgb_feature)
    labels.append(label)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [05:20<00:00,  1.56it/s]

CPU times: total: 4min 26s
Wall time: 5min 20s


In [5]:
X_tr_2= np.array(X_tr)
print(X_tr_2.shape)

(1000, 4096)


In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np.array(labels)
labels = to_categorical(labels)
print(le.classes_, labels.shape)

['NonViolence' 'Violence'] (1000, 2)


In [12]:
from sklearn.model_selection import train_test_split

(trainX, testX, trainY, testY) = train_test_split(X_tr_2, labels,test_size=0.1, random_state=40)

In [13]:
print(trainX.shape,testX.shape, trainY.shape, testY.shape)

(900, 4096) (100, 4096) (900, 2) (100, 2)


In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

# MODEL MLP

In [34]:
model = Sequential()
model.add(Dense(256, input_dim=4096, kernel_initializer='glorot_normal', kernel_regularizer=l2(0.01), activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(64, kernel_initializer='glorot_normal',  activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(2, kernel_initializer='glorot_normal',  activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 256)               1048832   
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                16448     
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 2)                 130       
                                                                 
Total params: 1,065,410
Trainable params: 1,065,410
Non-trainable params: 0
_________________________________________________________________


In [35]:
H = model.fit(trainX, trainY, batch_size=512, validation_data=(testX, testY),epochs = 200)

Epoch 1/200
2/2 [==============================] - 0s 137ms/step - loss: 6.5937 - accuracy: 0.5656 - val_loss: 5.0875 - val_accuracy: 0.8400
Epoch 2/200
2/2 [==============================] - 0s 40ms/step - loss: 5.4071 - accuracy: 0.7722 - val_loss: 4.9647 - val_accuracy: 0.8400
Epoch 3/200
2/2 [==============================] - 0s 38ms/step - loss: 5.0691 - accuracy: 0.8456 - val_loss: 4.8369 - val_accuracy: 0.8600
Epoch 4/200
2/2 [==============================] - 0s 38ms/step - loss: 4.8928 - accuracy: 0.8900 - val_loss: 4.6970 - val_accuracy: 0.8700
Epoch 5/200
2/2 [==============================] - 0s 36ms/step - loss: 4.8062 - accuracy: 0.8989 - val_loss: 4.5608 - val_accuracy: 0.8700
Epoch 6/200
2/2 [==============================] - 0s 35ms/step - loss: 4.6136 - accuracy: 0.9122 - val_loss: 4.4195 - val_accuracy: 0.9100
Epoch 7/200
2/2 [==============================] - 0s 36ms/step - loss: 4.4102 - accuracy: 0.9189 - val_loss: 4.2762 - val_accuracy: 0.9200
Epoch 8/200
2/2 [==

2/2 [==============================] - 0s 29ms/step - loss: 0.4644 - accuracy: 0.9989 - val_loss: 0.6236 - val_accuracy: 0.9400
Epoch 60/200
2/2 [==============================] - 0s 29ms/step - loss: 0.4550 - accuracy: 0.9978 - val_loss: 0.5684 - val_accuracy: 0.9400
Epoch 61/200
2/2 [==============================] - 0s 28ms/step - loss: 0.4272 - accuracy: 0.9989 - val_loss: 0.5075 - val_accuracy: 0.9500
Epoch 62/200
2/2 [==============================] - 0s 28ms/step - loss: 0.4237 - accuracy: 0.9956 - val_loss: 0.4777 - val_accuracy: 0.9500
Epoch 63/200
2/2 [==============================] - 0s 29ms/step - loss: 0.4042 - accuracy: 0.9911 - val_loss: 0.4713 - val_accuracy: 0.9500
Epoch 64/200
2/2 [==============================] - 0s 30ms/step - loss: 0.3949 - accuracy: 0.9933 - val_loss: 0.5053 - val_accuracy: 0.9400
Epoch 65/200
2/2 [==============================] - 0s 30ms/step - loss: 0.3658 - accuracy: 0.9978 - val_loss: 0.5144 - val_accuracy: 0.9400
Epoch 66/200
2/2 [========

2/2 [==============================] - 0s 51ms/step - loss: 0.1603 - accuracy: 0.9922 - val_loss: 0.3051 - val_accuracy: 0.9300
Epoch 118/200
2/2 [==============================] - 0s 44ms/step - loss: 0.1370 - accuracy: 0.9989 - val_loss: 0.2911 - val_accuracy: 0.9300
Epoch 119/200
2/2 [==============================] - 0s 44ms/step - loss: 0.1407 - accuracy: 0.9956 - val_loss: 0.2848 - val_accuracy: 0.9300
Epoch 120/200
2/2 [==============================] - 0s 39ms/step - loss: 0.1504 - accuracy: 0.9967 - val_loss: 0.2835 - val_accuracy: 0.9300
Epoch 121/200
2/2 [==============================] - 0s 35ms/step - loss: 0.1478 - accuracy: 0.9922 - val_loss: 0.2728 - val_accuracy: 0.9300
Epoch 122/200
2/2 [==============================] - 0s 35ms/step - loss: 0.1409 - accuracy: 0.9933 - val_loss: 0.3050 - val_accuracy: 0.9400
Epoch 123/200
2/2 [==============================] - 0s 35ms/step - loss: 0.1537 - accuracy: 0.9889 - val_loss: 0.2556 - val_accuracy: 0.9500
Epoch 124/200
2/2 [=

Epoch 175/200
2/2 [==============================] - 0s 33ms/step - loss: 0.1293 - accuracy: 0.9900 - val_loss: 0.3284 - val_accuracy: 0.9300
Epoch 176/200
2/2 [==============================] - 0s 30ms/step - loss: 0.1148 - accuracy: 0.9911 - val_loss: 0.2283 - val_accuracy: 0.9400
Epoch 177/200
2/2 [==============================] - 0s 33ms/step - loss: 0.1048 - accuracy: 0.9933 - val_loss: 0.1934 - val_accuracy: 0.9600
Epoch 178/200
2/2 [==============================] - 0s 34ms/step - loss: 0.0984 - accuracy: 0.9933 - val_loss: 0.2124 - val_accuracy: 0.9400
Epoch 179/200
2/2 [==============================] - 0s 37ms/step - loss: 0.1101 - accuracy: 0.9922 - val_loss: 0.3291 - val_accuracy: 0.9300
Epoch 180/200
2/2 [==============================] - 0s 37ms/step - loss: 0.1135 - accuracy: 0.9922 - val_loss: 0.2182 - val_accuracy: 0.9300
Epoch 181/200
2/2 [==============================] - 0s 33ms/step - loss: 0.1214 - accuracy: 0.9900 - val_loss: 0.2000 - val_accuracy: 0.9600
Epoch 

In [21]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

In [23]:
plt.style.use("ggplot")
plt.figure()
N = 100
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="test_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="test_acc")
plt.title("Training Loss and Accuracy on Santa/Not Santa")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot")

In [ ]:
model.save("check.h5")

In [46]:
video = "test/pokemon.mp4"

cap = cv2.VideoCapture(video)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames = 512
frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
frame_numbers.sort()
frames = []
for frame_number in frame_numbers:    
    ret, frame = cap.read()        
    if frame is None:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(480,640))
    frames.append(frame)
input=np.array(frames)
clip = preprocess_input(input)
rgb_feature = feature_extractor.predict(clip)[0]
rgb_feature = np.expand_dims(rgb_feature, axis=0)
pred = model.predict(rgb_feature)[0]
print(le.inverse_transform([np.argmax(pred)])[0], pred[np.argmax(pred)], pred)            

NonViolence 0.9999833 [9.9998331e-01 1.6684977e-05]


In [28]:
le.classes_

array(['NonViolence', 'Violence'], dtype='<U11')